In [1]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from PIL import Image
import numpy as np
import os

# Spark Session oluşturma
spark = SparkSession.builder.master("local").appName("LSHExample").getOrCreate()

# Veri setinin bulunduğu dizin
data_dir = 'C:/Users/enver/Desktop/flw'

# Veri setinden özellik vektörlerini ve dosya yollarını oluşturma
data = []

# Alt klasörleri dolaşma
for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)
    
    # Sadece klasörleri kontrol etme
    if os.path.isdir(subdir_path):
        for img_file in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_file)
            img = Image.open(img_path)
            img_array = np.array(img)
            img_vector = Vectors.dense(img_array.flatten())
            data.append((img_vector, img_path))

df = spark.createDataFrame(data, ["features", "image_path"])



In [2]:
# LSH modelini oluşturma
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", numHashTables=5)
model = brp.fit(df)

# Referans resmi için özellik vektörünü oluşturma
reference_img_path = '‪C:/Users/enver/Documents/1111.jpeg'
reference_img = Image.open(reference_img_path)
reference_img = reference_img.resize((250, 250))  # Giriş boyutlarına uygun olarak ayarlayın
reference_img_array = np.array(reference_img)
reference_img_vector = Vectors.dense(reference_img_array.flatten())

# Referans resmi ile en yakın 5 resmi bulma
reference_df = spark.createDataFrame([(reference_img_vector, "reference_image")], ["features", "image_path"])
similar_images = model.approxSimilarityJoin(df, reference_df, 0.5, distCol="JaccardDistance")

# En yakın 5 resmi yazdırma
closest_images = similar_images.select("datasetB.image_path").limit(5)
closest_images.show(truncate=False)


Py4JJavaError: An error occurred while calling o44.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (Mergen executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)